**Construct and export a MySQL database using the previously stored data**

# Imports and Data Loading

In [4]:
#Importing all of the libraries that may be needed for the project
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from urllib.parse import quote  
from sqlalchemy.engine import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy_utils import create_database, database_exists 

# Design the ERD in MySQL

![png](ERD_v2.png)

Tables have been created using forward engineering, the next steps will consist of using python to update these tables

# Data Cleaning

## TMDB Data

In [7]:
#Loading the data
tmdb = pd.read_csv('Data/tmdb_results_combined.csv.gz', low_memory=False)
tmdb.head(2)

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
1,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.104,2029.0,PG


In [8]:
tmdb_simplified = tmdb[['imdb_id', 'revenue', 'budget' , 'certification']]
tmdb_simplified.head()

,imdb_id,revenue,budget,certification
0,tt0113026,0.0,10000000.0,NaN
1,tt0118694,12854953.0,150000.0,PG
2,tt0119273,0.0,15000000.0,R
3,tt0120263,80334.0,0.0,NaN
4,tt0120467,14904.0,120000.0,R


## Title Ratings

In [22]:
#Loading the data
ratings = pd.read_csv('Data/cleaned_ratings.csv.gz', low_memory=False)
ratings.head(2)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1923
1,tt0000002,5.8,259


In [23]:
#Renaming the columns
ratings = ratings.rename(columns={'tconst': 'movie_id',
                        'averageRating': 'average_movie_rating', 
                        'numVotes': 'number_of_votes'})
# DataFrame after renaming columns
ratings.head() 

,movie_id,average_movie_rating,number_of_votes
0,tt0000001,5.7,1923
1,tt0000002,5.8,259
2,tt0000005,6.2,2549
3,tt0000006,5.1,175
4,tt0000007,5.4,798


In [29]:
#adding rating id column
#ratings['rating_id'] = ratings['movie_id'] 
ratings['rating_id'] = range(1,len(ratings)+1,1)
ratings['rating_id']

0              1
1              2
2              3
3              4
4              5
           ...  
475959    475960
475960    475961
475961    475962
475962    475963
475963    475964
Name: rating_id, Length: 475964, dtype: int64

In [30]:
ratings.head()

,movie_id,average_movie_rating,number_of_votes,rating_id
0,tt0000001,5.7,1923,1
1,tt0000002,5.8,259,2
2,tt0000005,6.2,2549,3
3,tt0000006,5.1,175,4
4,tt0000007,5.4,798,5


In [33]:
#re-arranging the columns
cols = ['rating_id', 'average_movie_rating', 'number_of_votes', 'movie_id']
ratings_simplified = ratings[cols]
ratings_simplified.head()

,rating_id,average_movie_rating,number_of_votes,movie_id
0,1,5.7,1923,tt0000001
1,2,5.8,259,tt0000002
2,3,6.2,2549,tt0000005
3,4,5.1,175,tt0000006
4,5,5.4,798,tt0000007


## Title basics

In [35]:
#Loading the data
basics = pd.read_csv('Data/cleaned_basics.csv.gz', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [36]:
#renaming the columns
basics = basics.rename(columns={'tconst': 'movie_id',
                        'primaryTitle': 'primary_title',
                        'startYear': 'start_year',        
                        'runtimeMinutes': 'runtime_min'})
# DataFrame after renaming columns 
basics.head()

,movie_id,titleType,primary_title,originalTitle,isAdult,start_year,endYear,runtime_min,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [37]:
basics_simplified = basics[['movie_id', 'primary_title', 'start_year', 'runtime_min']]
basics_simplified.head()

,movie_id,primary_title,start_year,runtime_min
0,tt0035423,Kate & Leopold,2001.0,118
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020.0,70
2,tt0069049,The Other Side of the Wind,2018.0,122
3,tt0088751,The Naked Monster,2005.0,100
4,tt0096056,Crime and Punishment,2002.0,126


## Title Genres

In [38]:
genres_df = basics[['movie_id', 'genres']]
genres_df.head()

,movie_id,genres
0,tt0035423,"Comedy,Fantasy,Romance"
1,tt0062336,Drama
2,tt0069049,Drama
3,tt0088751,"Comedy,Horror,Sci-Fi"
4,tt0096056,Drama


In [39]:
## create a col with a list of genres
genres_df['genres_split'] = genres_df['genres'].str.split(',')
genres_df.head()

C:\Users\akinde.kadjo\AppData\Local\Temp\ipykernel_26044\1268187554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres_df['genres_split'] = genres_df['genres'].str.split(',')


,movie_id,genres,genres_split
0,tt0035423,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,Drama,[Drama]
2,tt0069049,Drama,[Drama]
3,tt0088751,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
4,tt0096056,Drama,[Drama]


In [41]:
exploded_genres = genres_df.explode('genres_split')
exploded_genres.head()

,movie_id,genres,genres_split
0,tt0035423,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,"Comedy,Fantasy,Romance",Romance
1,tt0062336,Drama,Drama
2,tt0069049,Drama,Drama


In [42]:
#unique genres
unique_genres = sorted(exploded_genres['genres_split'].unique())
unique_genres

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

In [49]:
#saving the df for the title genres table
title_genres = exploded_genres[['movie_id', 'genres_split']]
title_genres = title_genres_simplified.rename(columns={'genres_split': 'genres_name'})
title_genres.head()

,movie_id,genres_name
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0069049,Drama


In [50]:
## Making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Short': 20,
 'Sport': 21,
 'Talk-Show': 22,
 'Thriller': 23,
 'War': 24,
 'Western': 25}

In [51]:
genres = title_genres[['movie_id','genres_name']]
genres.head()

,movie_id,genres_name
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0069049,Drama


In [52]:
## make new integer genre_id 
genres['genre_id'] = genres['genres_name'].map(genre_map)
genres.head()

,movie_id,genres_name,genre_id
0,tt0035423,Comedy,5
0,tt0035423,Fantasy,9
0,tt0035423,Romance,18
1,tt0062336,Drama,7
2,tt0069049,Drama,7


In [53]:
#final version
title_genres_simplified = genres[['movie_id','genre_id']]
title_genres_simplified. head()

,movie_id,genre_id
0,tt0035423,5
0,tt0035423,9
0,tt0035423,18
1,tt0062336,7
2,tt0069049,7


## Genres

In [54]:
#Convert the genre map dictionary into a dataframe
genres_simplified = pd.DataFrame({'genre_id':genre_map.values(),'genre_name':genre_map.keys() })
genres_simplified.head()

,genre_id,genre_name
0,0,Action
1,1,Adult
2,2,Adventure
3,3,Animation
4,4,Biography


# Converting Dataframes in MySQL tables

In [55]:
# connection
engine = create_engine('mysql+pymysql://root:%s@localhost:3306/movies' % quote('pwd'))
                                                                   
# Check if the database exists. If not, create it.
if database_exists('mysql+pymysql://root:%s@localhost:3306/movies' % quote('pwd')) == False:
  create_database('mysql+pymysql://root:%s@localhost:3306/movies' % quote('pwd'))
else:
  print('The database already exists')

The database already exists


In [79]:
from sqlalchemy.types import *
## Calculate max string lengths for object columns
key_len = basics_simplified['movie_id'].fillna('').map(len).max()
title_len = basics_simplified['primary_title'].fillna('').map(len).max()
## Create a schema dictonary using Sqlalchemy datatype objects
basics_simplified_schema = {
                            "movie_id": String(key_len+1), 
                            "primary_title": Text(title_len+1),
                            'start_year':Float(),
                            'runtime_min':Integer()}

In [80]:
# Save to sql with dtype and index=False
basics_simplified.to_sql('title_basics',engine,dtype=basics_simplified_schema,if_exists='replace',index=False)

83854

In [77]:
## Calculate max string lengths for object columns
key_len = tmdb_simplified['imdb_id'].fillna('').map(len).max()
cert_len = tmdb_simplified['certification'].fillna('').map(len).max()

tmdb_simplified_schema = {
                            "imdb_id":String(key_len+1), 
                            'revenue':Float(),
                            'budget':Float(),
                            "certification":Text(cert_len+1)}

In [78]:
# Save to sql with dtype and index=False
tmdb_simplified.to_sql('tmdb_data',engine,dtype=tmdb_simplified_schema,if_exists='replace',index=False)

625

In [81]:
#converting all the other dataframes to sql tables
ratings_simplified.to_sql('title_ratings', engine, if_exists = 'replace')
genres_simplified.to_sql('genres', engine, if_exists = 'replace')
title_genres_simplified.to_sql('title_genres', engine, if_exists = 'replace')

156688

In [82]:
#ADD PRIMARY KEY
engine.execute('ALTER TABLE title_basics ADD PRIMARY KEY (`movie_id`);')

In [83]:
engine.execute('ALTER TABLE title_ratings ADD PRIMARY KEY (`rating_id`);')

In [84]:
engine.execute('ALTER TABLE genres ADD PRIMARY KEY (`genre_id`);')

In [85]:
engine.execute('ALTER TABLE tmdb_data ADD PRIMARY KEY (`imdb_id`);')

Note: Index columns were dropped within mySQL

# SQL Queries

In [86]:
q = """SHOW TABLES;"""
pd.read_sql(q, engine)

,Tables_in_movies
0,genres
1,title_basics
2,title_genres
3,title_ratings
4,tmdb_data


In [87]:
#title_basics table
q="""
SELECT*
FROM title_basics
LIMIT 5;
"""
pd.read_sql(q, engine)

,movie_id,primary_title,start_year,runtime_min
0,tt0035423,Kate & Leopold,2001.0,118
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020.0,70
2,tt0069049,The Other Side of the Wind,2018.0,122
3,tt0088751,The Naked Monster,2005.0,100
4,tt0096056,Crime and Punishment,2002.0,126


In [88]:
#title_ratings table
q="""
SELECT*
FROM title_ratings
LIMIT 5;
"""
pd.read_sql(q, engine)

,rating_id,average_movie_rating,number_of_votes,movie_id
0,1,5.7,1923,tt0000001
1,2,5.8,259,tt0000002
2,3,6.2,2549,tt0000005
3,4,5.1,175,tt0000006
4,5,5.4,798,tt0000007


In [89]:
#tmdb_data table
q="""
SELECT*
FROM tmdb_data
LIMIT 5;
"""
pd.read_sql(q, engine)

,imdb_id,revenue,budget,certification
0,tt0035423,76019000.0,48000000.0,PG-13
1,tt0113026,0.0,10000000.0,None
2,tt0118589,5271670.0,22000000.0,PG-13
3,tt0118652,0.0,1000000.0,R
4,tt0118694,12855000.0,150000.0,PG


In [90]:
#genres table
q="""
SELECT*
FROM genres
LIMIT 5;
"""
pd.read_sql(q, engine)

,genre_id,genre_name
0,0,Action
1,1,Adult
2,2,Adventure
3,3,Animation
4,4,Biography


In [91]:
#title_genres table
q="""
SELECT*
FROM title_genres
LIMIT 5;
"""
pd.read_sql(q, engine)

,movie_id,genre_id
0,tt0035423,5
1,tt0035423,9
2,tt0035423,18
3,tt0062336,7
4,tt0069049,7
